In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("agaricus-lepiota.data", header = None)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       8124 non-null   object
 1   1       8124 non-null   object
 2   2       8124 non-null   object
 3   3       8124 non-null   object
 4   4       8124 non-null   object
 5   5       8124 non-null   object
 6   6       8124 non-null   object
 7   7       8124 non-null   object
 8   8       8124 non-null   object
 9   9       8124 non-null   object
 10  10      8124 non-null   object
 11  11      8124 non-null   object
 12  12      8124 non-null   object
 13  13      8124 non-null   object
 14  14      8124 non-null   object
 15  15      8124 non-null   object
 16  16      8124 non-null   object
 17  17      8124 non-null   object
 18  18      8124 non-null   object
 19  19      8124 non-null   object
 20  20      8124 non-null   object
 21  21      8124 non-null   object
 22  22      8124 non-null   

In [13]:
from sklearn.model_selection import train_test_split

df.columns = ['class', 'cap_shape', 'cap_surface', 'cap_color', 'bruises', 'odor', 
               'gill_attachment', 'gill_spacing', 'gill_size', 'gill_color', 
               'stalk_shape', 'stalk_root', 'stalk_surface_above_ring', 'stalk_surface_below_ring', 'stalk_color_above_ring', 'stalk_color_below_ring', 
               'veil_type', 'veil_color', 'ring_number', 'ring_type', 'spore_print_color', 'population', 'habitat']
df = df.drop(df[df.stalk_root  == "?"].index)
X = df.drop('class', axis=1)
y = df['class']


train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)



Node Class

In [7]:
class Node():
        def __init__(self, feature_index = None, threshold=None, left = None, right = None, info_gain = None, value = None):
                self.feature_index = feature_index
                self.threshold = threshold
                self.left = left
                self.right = right
                self.info_gain = info_gain


                self.value = value
        


Tree Class

In [15]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split = 2,max_depth = 2):
        self.root = None

        self.max_depth = max_depth
        self.min_samples_split = min_samples_split

    def build_tree(self, train_X, train_y, current_depth = 0):
        num_samples, num_features = train_X.shape()

        if num_samples >= self.min_samples_split and current_depth <= self.max_depth:

            best_split = self.get_best_split(train_X, train_y, num_samples, num_features)

            if best_split["info-gain"]>0:
                left_subtree = self.build_tree(best_split["X_left"], best_split["y_left"], current_depth + 1)
                right_subtree = self.build_tree(best_split["X_right"], best_split["y_right"], current_depth + 1)

                return Node(best_split["feature_index"], best_split["threshold"], left_subtree, right_subtree, best_split["info_gain"])

        leaf_value = self.calculate_leaf_value(y)
        return Node(value = leaf_value)
        
    def best_split(self, train_X, train_y, num_samples, num_features):

        best_split = {}
        max_info_gain = -float("inf")

        for feature_index in range(num_features):
            feature_values = train_X[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds:
                X_left, X_right, y_left, y_right = self.split(train_X, train_y, feature_index, threshold)


